## 首先，請先安裝必要的套件：

In [ ]:
!pip install langgraph openai

## 構建我們的 LangGraph 工作流程。

In [ ]:
# 1. 導入必要的模組與套件
from langgraph.graph import StateGraph, END
from dataclasses import dataclass
from typing import Dict
from openai import OpenAI
import os

In [ ]:
# 2. 定義「狀態（State）」結構：這是我們AI的記憶體
@dataclass
class ReActState:
    input: str
    history: list
    result: str = ""
    iteration: int = 0

In [ ]:
# 3. 初始化 OpenAI 客戶端 (請先設定好你的環境變數 OPENAI_API_KEY)
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [ ]:
# 4. 定義「節點（Nodes）」：這是AI執行的任務
def llm_call(prompt: str, max_tokens: int = 100) -> str:
    # 這是實際呼叫 OpenAI 模型的函式
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens
    )
    return response.choices[0].message.content.strip()

def generate_node(state: ReActState) -> Dict:
    """生成節點：負責將內容轉為正式用語"""
    # 這裡可以根據歷史記錄來優化提示
    prompt = f"請將以下內容轉為專業的會議記錄格式：\n{state.input}\n\n參考歷史紀錄來優化：{state.history}"
    generated_text = llm_call(prompt)
    
    # 更新狀態：儲存這次的生成結果與歷史
    state.result = generated_text
    state.history.append(f"第 {state.iteration + 1} 次生成：{generated_text}")
    return {"result": generated_text, "history": state.history}

def check_node(state: ReActState) -> Dict:
    """檢查節點：負責更新迭代次數"""
    state.iteration += 1
    return {"iteration": state.iteration}

In [ ]:
# 5. 構建「圖（Graph）」與「邊（Edges）」：這是AI的工作流程
workflow = StateGraph(ReActState)

# 新增節點到工作流程中
workflow.add_node("generate", generate_node)
workflow.add_node("check", check_node)

# 設定進入點：流程從 "generate" 節點開始
workflow.set_entry_point("generate")

# 設定邊：從 "generate" 節點無條件跳轉到 "check" 節點
workflow.add_edge("generate", "check")

# 設定條件邊：從 "check" 節點根據條件跳轉
workflow.add_conditional_edges(
    "check",
    # 判斷函數：如果迭代次數 >= 2 就結束，否則回到 "generate"
    lambda state: "end" if state.iteration >= 2 else "generate",
    {"generate": "generate", "end": END}
)

# 編譯工作流程，準備執行
graph = workflow.compile()


## 執行範例與成果展示
現在，我們只需用一個簡單的輸入來啟動這個工作流程，LangGraph 就會自動執行我們設定好的節點與邊。

In [ ]:
# 執行範例
input_data = ReActState(input="會議討論了新產品發布，並探討了行銷策略。", history=[])
result = graph.invoke(input_data)

print(f"最終用語：{result['result']}")
print(f"生成歷史：{result['history']}")